In [2]:
import numpy as np
import pandas as pd
import twitter
import nltk
import re
import oauth2 as oauth
import tweepy
import time
import twint
import nest_asyncio
nest_asyncio.apply()

In [26]:
api_key = "E0kcW1Kx0GNc6doKP042WBVhA"
api_secret = "FlWj4g7Vtbi6UKiFDSdMeSwTNnva2mG8DWEdgCKLX8t0PDyRGl"
access_token_key = "1403330200285454340-JZnlzgjjc4TNNkGgeEgv6UnaVehKX3"
access_token_secret = "uFYlateFrodReYw8a1hjt7YyQDFMM6g5fJ273P2e6tR77"

_debug = 0

oauth_token = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

In [27]:
username = '@NorbertBolz'
count = 100
try:     
    tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)
    tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
    tweets_df = pd.DataFrame(tweets_list)
    
except BaseException as e:
      print('failed on_status,',str(e))
      time.sleep(3)

C:\Users\sched\anaconda\lib\codeop.py:141: RuntimeWarning: coroutine 'Twint.main' was never awaited
  codeob = compile(source, filename, symbol, self.flags, 1)


failed on_status, name 'api' is not defined


In [ ]:
c = twint.Config()
#c.Username = '@NorbertBolz'
c.Search = "covid"
#c.Output = 'bolz.txt'

twint.run.Search(c)

In [ ]:
c = twint.Config()
c.Search = "covid"

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
